In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key="", base_url="https://api.deepseek.com")#Your model API key here

response = client.chat.completions.create(
    model="deepseek-reasoner",
    timeout= 1200,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Solve this puzzle for me.On a one-dimensional board, there are red checkers (’R’), blue checkers (’B’), and one empty space (’_’). A checker can move by either:1. Sliding forward into an adjacent empty space, or jumping over exactly one checker of the opposite color to land in an empty space.The goal is to swap the positions of all red and blue checkers, effectively mirroring the initialstate.Example: If the initial state is [’R’, ’_’, ’B’], the goal is to reach [’B’, ’_’, ’R’]. Your solutionshould be a list of moves where each move is represented as [checker_color, position_from,position_to]. For example:moves = [[ ’R ’ , 0 , 1] , [ ’B ’ , 2 , 0] , [ ’R ’, 1 , 2]]This means: Move the red checker from position 0 to 1, then move the blue checker fromposition 2 to 0, and so on.Requirements:• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.• The positions are 0-indexed (the leftmost position is 0).• Ensure your final answer includes the complete list of moves for final solution in theformat: moves = [[checker_color, position_from, position_to], ...]"},
        {"role": "user", "content": "I have a puzzle with 19 positions, where 9 red checkers (’R’) on left, 9 blue checkers(’B’) on right, and one empty space (’_’) in between are arranged in a line.Initial board: R R ... R _ B B ... BGoal board: B B ... B _ R R ... RRules:• A checker can slide into an adjacent empty space.• A checker can jump over exactly one checker of the opposite color to land in an emptyspace.• Checkers cannot move backwards (towards their starting side).Find the minimum sequence of moves to transform the initial board into the goal board."},
    ],
    stream=False
)

print(response.choices[0].message.content)

The minimum sequence of moves to swap the positions of the 9 red and 9 blue checkers on a 19-position board involves 99 moves. The solution follows a systematic pattern where moves alternate between sliding and jumping, ensuring no backward movement. Each move is represented as `[checker_color, position_from, position_to]`.

**Complete Move Sequence (99 moves):**
```
moves = [
    ['B', 10, 9], ['R', 8, 10], ['B', 9, 8], ['B', 11, 9], ['R', 7, 8], ['R', 5, 7], 
    ['B', 9, 5], ['B', 12, 9], ['B', 13, 12], ['R', 6, 7], ['R', 4, 6], ['R', 2, 4], 
    ['B', 5, 2], ['B', 9, 5], ['B', 14, 9], ['B', 15, 14], ['B', 16, 15], ['R', 7, 9], 
    ['R', 3, 7], ['R', 1, 3], ['R', 0, 1], ['B', 2, 0], ['B', 5, 2], ['B', 9, 5], 
    ['B', 17, 9], ['B', 18, 17], ['R', 10, 12], ['R', 8, 10], ['R', 6, 8], ['R', 4, 6], 
    ['R', 3, 4], ['B', 0, 3], ['B', 2, 0], ['B', 5, 2], ['B', 9, 5], ['B', 17, 9], 
    ['R', 12, 14], ['R', 10, 12], ['R', 8, 10], ['R', 7, 8], ['B', 3, 7], ['B', 0, 3], 
    ['B', 2, 0],

In [ ]:
from typing import List, Tuple

Move = Tuple[str, int, int]   # ('R' | 'B', src_index, dst_index)

class CheckerJumpingValidator:
    """
    Validate a 1-D checker-jumping solution sequence.

    Parameters
    ----------
    n : int
        Number of red checkers on the left / blue checkers on the right.
        The board length is 2*n + 1 (one extra for the empty slot).
    """

    def __init__(self, n: int) -> None:
        if n <= 0:
            raise ValueError("n must be positive")
        self.n = n
        self.board = ['R'] * n + ['_'] + ['B'] * n
        self.goal  = ['B'] * n + ['_'] + ['R'] * n

    # ------------------------------------------------------------
    # public interface
    # ------------------------------------------------------------
    def validate(self, moves: List[Move]) -> Tuple[bool, str]:
        """
        Validate the entire sequence.

        Returns
        -------
        (ok: bool, message: str)
            ok == True  → 全部合法且到达终局
            ok == False → message 里指出第一处错误
        """
        for step, move in enumerate(moves, 1):
            ok, msg = self._apply_one_move(*move)
            if not ok:
                return False, f"Step {step}: {msg}"
        if self.board != self.goal:
            return False, "Board does not match goal configuration"
        return True, "OK"

    # ------------------------------------------------------------
    # helpers
    # ------------------------------------------------------------
    def _apply_one_move(self, colour: str, src: int, dst: int) -> Tuple[bool, str]:
        colour = colour.upper()
        if colour not in {"R", "B"}:
            return False, f"Illegal colour {colour!r}"

        if not (0 <= src < len(self.board)) or not (0 <= dst < len(self.board)):
            return False, "Source/Destination index out of range"

        if self.board[src] != colour:
            return False, f"Source {src} does not contain {colour}"

        if self.board[dst] != '_':
            return False, f"Destination {dst} is not empty"

        delta = dst - src

        # direction rule
        if colour == 'R' and delta <= 0:
            return False, "Red checker must move to the right"
        if colour == 'B' and delta >= 0:
            return False, "Blue checker must move to the left"

        # slide or jump
        if abs(delta) == 1:           # slide
            pass
        elif abs(delta) == 2:         # jump
            mid = (src + dst) // 2
            mid_piece = self.board[mid]
            if mid_piece == '_' or mid_piece == colour:
                return False, "Invalid jump: no opponent piece in the middle"
        else:
            return False, "Illegal move distance (must be 1 or 2)"

        # apply move
        self.board[dst], self.board[src] = self.board[src], '_'
        return True, "move OK"


# ------------------------------------------------------------
# example usage
# ------------------------------------------------------------
if __name__ == "__main__":
    # 一个合法的 n=5（左右各 5 棋）最短 35 步解；仅作演示
    moves = [
        ['B', 10, 9], ['R', 8, 10], ['B', 9, 8], ['B', 11, 9], ['R', 7, 8], ['R', 5, 7],
        ['B', 9, 5], ['B', 12, 9], ['B', 13, 12], ['R', 6, 7], ['R', 4, 6], ['R', 2, 4],
        ['B', 5, 2], ['B', 9, 5], ['B', 14, 9], ['B', 15, 14], ['B', 16, 15], ['R', 7, 9],
        ['R', 3, 7], ['R', 1, 3], ['R', 0, 1], ['B', 2, 0], ['B', 5, 2], ['B', 9, 5],
        ['B', 17, 9], ['B', 18, 17], ['R', 10, 12], ['R', 8, 10], ['R', 6, 8], ['R', 4, 6],
        ['R', 3, 4], ['B', 0, 3], ['B', 2, 0], ['B', 5, 2], ['B', 9, 5], ['B', 17, 9],
        ['R', 12, 14], ['R', 10, 12], ['R', 8, 10], ['R', 7, 8], ['B', 3, 7], ['B', 0, 3],
        ['B', 2, 0], ['B', 5, 2], ['B', 9, 5], ['R', 14, 16], ['R', 12, 14], ['R', 11, 12],
        ['B', 7, 11], ['B', 3, 7], ['B', 0, 3], ['B', 2, 0], ['B', 5, 2], ['R', 16, 18],
        ['R', 14, 16], ['R', 13, 14], ['B', 11, 13], ['B', 7, 11], ['B', 3, 7], ['B', 0, 3],
        ['B', 2, 0], ['R', 18, 17], ['R', 16, 18], ['R', 15, 16], ['R', 14, 15], ['B', 13, 14],
        ['B', 11, 13], ['B', 7, 11], ['B', 6, 7], ['R', 17, 16], ['R', 18, 17], ['B', 14, 18],
        ['B', 13, 14], ['B', 11, 13], ['B', 10, 11], ['R', 16, 15], ['R', 17, 16], ['B', 18, 17],
        ['B', 14, 18], ['B', 12, 14], ['B', 11, 12], ['R', 15, 13], ['R', 16, 15], ['B', 17, 16],
        ['B', 18, 17], ['R', 13, 14], ['R', 15, 13], ['B', 16, 15], ['B', 17, 16], ['R', 14, 12],
        ['R', 13, 14], ['B', 15, 13], ['B', 16, 15], ['R', 12, 11], ['R', 14, 12], ['B', 13, 14],
        ['B', 15, 13], ['R', 11, 10], ['R', 12, 11], ['B', 14, 12], ['B', 13, 14], ['R', 10, 9]
    ]

    validator = CheckerJumpingValidator(n=9)
    ok, message = validator.validate(moves)
    print("Result:", ok, "-", message)


Result: False - Step 5: Destination 8 is not empty
